# Lab 4
## Build your own network

In this lab, you'll use command-line tools to build a simple application in JavaScript that queries and updates a prepopulated blockchain network ledger. Your application will use APIs to invoke smart contracts, or chaincode. You'll use the Hyperledger Fabric Node SDK to access the APIs.<br>

You'll work with networks in more detail in the next lab.<br>

Notice for "Writing Your First Application" Module: Copyright Hyperledger 2017. Revision b33fb974. Licensed under Creative Commons Attribution 4.0 International License https://creativecommons.org/licenses/by/4.0/legalcode. 
* Module available at http://hyperledger-fabric.readthedocs.io/en/latest/write_first_app.html.

### [Start the lab!](http://hyperledger-fabric.readthedocs.io/en/latest/write_first_app.html)
<br><br>


The build your first network (BYFN) scenario provisions a sample Hyperledger Fabric network consisting of two organizations, each maintaining two peer nodes, and a “solo” ordering service.

### Install prerequisites

Before we begin, if you haven’t already done so, you may wish to check that you have all the [Prerequisites](http://hyperledger-fabric.readthedocs.io/en/latest/prereqs.html) installed on the platform(s) on which you’ll be developing blockchain applications and/or operating Hyperledger Fabric.<br>

You will also need to download and install the [Hyperledger Fabric Samples](http://hyperledger-fabric.readthedocs.io/en/latest/samples.html). You will notice that there are a number of samples included in the `fabric-samples` repository. We will be using the `first-network` sample. Let’s open that sub-directory now.<br>

```bash
cd fabric-samples/first-network
```

## Want to run it now?

We provide a fully annotated script - `byfn.sh` - that leverages these Docker images to quickly bootstrap a Hyperledger Fabric network comprised of 4 peers representing two different organizations, and an orderer node. It will also launch a container to run a scripted execution that will join peers to a channel, deploy and instantiate chaincode and drive execution of transactions against the deployed chaincode.<br>

Here’s the help text for the `byfn.sh` script:

```bash
./byfn.sh --help
Usage:
byfn.sh -m up|down|restart|generate [-c <channel name>] [-t <timeout>] [-d <delay>] [-f <docker-compose-file>] [-s <dbtype>]
byfn.sh -h|--help (print this message)
  -m <mode> - one of 'up', 'down', 'restart' or 'generate'
    - 'up' - bring up the network with docker-compose up
    - 'down' - clear the network with docker-compose down
    - 'restart' - restart the network
    - 'generate' - generate required certificates and genesis block
  -c <channel name> - channel name to use (defaults to "mychannel")
  -t <timeout> - CLI timeout duration in seconds (defaults to 10000)
  -d <delay> - delay duration in seconds (defaults to 3)
  -f <docker-compose-file> - specify which docker-compose file use (defaults to docker-compose-cli.yaml)
  -s <dbtype> - the database backend to use: goleveldb (default) or couchdb
  -l <language> - the chaincode language: golang (default) or node

  Typically, one would first generate the required certificates and
  genesis block, then bring up the network. e.g.:

      byfn.sh -m generate -c mychannel
      byfn.sh -m up -c mychannel -s couchdb
```

If you choose not to supply a channel name, then the script will use a default name of `mychannel`. The CLI timeout parameter (specified with the -t flag) is an optional value; if you choose not to set it, then your CLI container will exit after the default setting of 10000 seconds.


## Generate Network Artifacts

Ready to give it a go? Okay then! Execute the following command:

```
./byfn.sh -m generate
```

You will see a brief description as to what will occur, along with a yes/no command line prompt. Respond with a `y` to execute the described action.

```
Generating certs and genesis block for with channel 'mychannel' and CLI timeout of '10000'
Continue (y/n)?y
proceeding ...
/Users/xxx/dev/fabric-samples/bin/cryptogen

##########################################################
##### Generate certificates using cryptogen tool #########
##########################################################
org1.example.com
2017-06-12 21:01:37.334 EDT [bccsp] GetDefault -> WARN 001 Before using BCCSP, please call InitFactories(). Falling back to bootBCCSP.
...

/Users/xxx/dev/fabric-samples/bin/configtxgen
##########################################################
#########  Generating Orderer Genesis block ##############
##########################################################
2017-06-12 21:01:37.558 EDT [common/configtx/tool] main -> INFO 001 Loading configuration
2017-06-12 21:01:37.562 EDT [msp] getMspConfig -> INFO 002 intermediate certs folder not found at [/Users/xxx/dev/byfn/crypto-config/ordererOrganizations/example.com/msp/intermediatecerts]. Skipping.: [stat /Users/xxx/dev/byfn/crypto-config/ordererOrganizations/example.com/msp/intermediatecerts: no such file or directory]
...
2017-06-12 21:01:37.588 EDT [common/configtx/tool] doOutputBlock -> INFO 00b Generating genesis block
2017-06-12 21:01:37.590 EDT [common/configtx/tool] doOutputBlock -> INFO 00c Writing genesis block

#################################################################
### Generating channel configuration transaction 'channel.tx' ###
#################################################################
2017-06-12 21:01:37.634 EDT [common/configtx/tool] main -> INFO 001 Loading configuration
2017-06-12 21:01:37.644 EDT [common/configtx/tool] doOutputChannelCreateTx -> INFO 002 Generating new channel configtx
2017-06-12 21:01:37.645 EDT [common/configtx/tool] doOutputChannelCreateTx -> INFO 003 Writing new channel tx

#################################################################
#######    Generating anchor peer update for Org1MSP   ##########
#################################################################
2017-06-12 21:01:37.674 EDT [common/configtx/tool] main -> INFO 001 Loading configuration
2017-06-12 21:01:37.678 EDT [common/configtx/tool] doOutputAnchorPeersUpdate -> INFO 002 Generating anchor peer update
2017-06-12 21:01:37.679 EDT [common/configtx/tool] doOutputAnchorPeersUpdate -> INFO 003 Writing anchor peer update

#################################################################
#######    Generating anchor peer update for Org2MSP   ##########
#################################################################
2017-06-12 21:01:37.700 EDT [common/configtx/tool] main -> INFO 001 Loading configuration
2017-06-12 21:01:37.704 EDT [common/configtx/tool] doOutputAnchorPeersUpdate -> INFO 002 Generating anchor peer update
2017-06-12 21:01:37.704 EDT [common/configtx/tool] doOutputAnchorPeersUpdate -> INFO 003 Writing anchor peer update
```

This first step generates all of the certificates and keys for our various network entities, the **`genesis block`** used to bootstrap the ordering service, and a collection of configuration transactions required to configure a [Channel](http://hyperledger-fabric.readthedocs.io/en/latest/glossary.html#channel).

## Bringing Up the Network
Next, you can bring the network up with one of the following commands:

```
./byfn.sh -m up
```
The above command will **compile Golang chaincode images** and **spin up the corresponding containers**. 
* Go is the default chaincode language, however there is also support for Node.js chaincode. If you’d like to run through this tutorial with node chaincode, pass the following command instead:

```
# we use the -l flag to specify the chaincode language
# forgoing the -l flag will default to Golang

./byfn.sh -m up -l node
```
#### NOTE
View the [Hyperledger Fabric Shim](https://fabric-shim.github.io/ChaincodeStub.html) documentation for more info on the node.js chaincode shim APIs.<br>

### Troubleshoot
* `cryptogen tool not found. exiting`
* https://stackoverflow.com/questions/45033956/how-do-i-resolve-the-error-on-hyperledger-fabric
  * In the current environment I use<br>
  ```cd fabric-samples
    curl -sSL https://raw.githubusercontent.com/hyperledger/fabric/master/scripts/bootstrap-1.0.0.sh | bash```

Once again, you will be prompted as to whether you wish to continue or abort. Respond with a `y`:

```
Starting with channel 'mychannel' and CLI timeout of '10000'
Continue (y/n)?y
proceeding ...
Creating network "net_byfn" with the default driver
Creating peer0.org1.example.com
Creating peer1.org1.example.com
Creating peer0.org2.example.com
Creating orderer.example.com
Creating peer1.org2.example.com
Creating cli


 ____    _____      _      ____    _____
/ ___|  |_   _|    / \    |  _ \  |_   _|
\___ \    | |     / _ \   | |_) |   | |
 ___) |   | |    / ___ \  |  _ <    | |
|____/    |_|   /_/   \_\ |_| \_\   |_|

Channel name : mychannel
Creating channel...
```
The logs will continue from there. This will launch all of the containers, and then drive a complete end-to-end application scenario. Upon successful completion, it should report the following in your terminal window:

```
2017-05-16 17:08:01.366 UTC [msp] GetLocalMSP -> DEBU 004 Returning existing local MSP
2017-05-16 17:08:01.366 UTC [msp] GetDefaultSigningIdentity -> DEBU 005 Obtaining default signing identity
2017-05-16 17:08:01.366 UTC [msp/identity] Sign -> DEBU 006 Sign: plaintext: 0AB1070A6708031A0C08F1E3ECC80510...6D7963631A0A0A0571756572790A0161
2017-05-16 17:08:01.367 UTC [msp/identity] Sign -> DEBU 007 Sign: digest: E61DB37F4E8B0D32C9FE10E3936BA9B8CD278FAA1F3320B08712164248285C54
Query Result: 90
2017-05-16 17:08:15.158 UTC [main] main -> INFO 008 Exiting.....
===================== Query on PEER3 on channel 'mychannel' is successful =====================

===================== All GOOD, BYFN execution completed =====================


 _____   _   _   ____
| ____| | \ | | |  _ \
|  _|   |  \| | | | | |
| |___  | |\  | | |_| |
|_____| |_| \_| |____/
```

You can scroll through these logs to see the various transactions. If you don’t get this result, then jump down to the [Troubleshooting](http://hyperledger-fabric.readthedocs.io/en/latest/build_network.html#troubleshoot) section and let’s see whether we can help you discover what went wrong.